In [1]:
# Gerekli kütüphanelerin tanımlanması ve ortam ayarları
import os
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib  # Model kaydetme işlemi için

# Scikit-learn modülleri
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA  
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import silhouette_score, davies_bouldin_score

# Uyarıları filtrele
warnings.filterwarnings('ignore')

# Windows işlemci çekirdek hatası için düzeltme (Joblib/Loky)
os.environ["LOKY_MAX_CPU_COUNT"] = "4"

print("Kütüphaneler yüklendi.")

Kütüphaneler yüklendi ve ortam Sadek Ali Elzazkouk için hazır.


In [2]:
# Veri setinin yüklenmesi
try:
    df = pd.read_csv('owid-energy-data.csv')
except FileNotFoundError:
    print("Hata: 'owid-energy-data.csv' bulunamadı.")

# 2022 yılı verilerinin filtrelenmesi
df_2022 = df[df['year'] == 2022].copy()
df_clean = df_2022.dropna(subset=['iso_code'])
df_clean = df_clean[~df_clean['iso_code'].str.contains('OWID')]

# Analiz özellikleri
features = [
    'biofuel_share_elec', 'coal_share_elec', 'solar_share_elec',
    'hydro_share_elec', 'nuclear_share_elec', 'oil_share_elec',
    'gas_share_elec', 'wind_share_elec', 'carbon_intensity_elec',
    'gdp', 'population', 'energy_per_capita'
]

data = df_clean[features].copy()

# Eksik verilerin medyan ile doldurulması
for col in data.columns:
    medyan = data[col].median()
    data[col].fillna(medyan, inplace=True)

# Standardizasyon (Z-Skor)
# PCA varyansa duyarlı olduğu için bu adım kritiktir.
scaler = StandardScaler()
X_scaled = scaler.fit_transform(data)

# İşlenmiş veriyi DataFrame yapısı
X_df = pd.DataFrame(X_scaled, columns=data.columns)

print(f"Veri ön işleme tamamlandı. Gözlem sayısı: {len(X_df)}")

Veri ön işleme tamamlandı. Gözlem sayısı: 220


In [3]:
# --- Yöntem B: Boyut İndirgeme / PCA ---

# Varyansın %90'ını açıklayan bileşen sayısını otomatik seç
pca = PCA(n_components=0.90)
X_pca = pca.fit_transform(X_df)

print(f"PCA işlemi tamamlandı.")
print(f"Orijinal Özellik Sayısı: {X_df.shape[1]}")
print(f"PCA Sonrası Bileşen Sayısı: {pca.n_components_}")
print(f"Toplam Açıklanan Varyans Oranı: {np.sum(pca.explained_variance_ratio_):.2f}")

PCA işlemi tamamlandı.
Orijinal Özellik Sayısı: 12
PCA Sonrası Bileşen Sayısı: 8
Toplam Açıklanan Varyans Oranı: 0.93


In [4]:
# --- Base Modellerin Kurulumu ve Eğitimi (PCA Verisi ile) ---

k = 3  # Küme sayısı

# 1. Model: K-Means (PCA verisi kullanılır)
kmeans_model = KMeans(n_clusters=k, random_state=42, n_init=10)
kmeans_labels = kmeans_model.fit_predict(X_pca)

# 2. Model: Agglomerative Hierarchical Clustering (PCA verisi kullanılır)
agg_model = AgglomerativeClustering(n_clusters=k)
agg_labels = agg_model.fit_predict(X_pca)

# Başarı Metrikleri
results = {
    "K-Means Silhouette": silhouette_score(X_pca, kmeans_labels),
    "K-Means Davies-Bouldin": davies_bouldin_score(X_pca, kmeans_labels),
    "Hierarchical Silhouette": silhouette_score(X_pca, agg_labels),
    "Hierarchical Davies-Bouldin": davies_bouldin_score(X_pca, agg_labels)
}

print("Modeller PCA verisi üzerinde başarıyla eğitildi.\n")
print("-" * 40)
print(f"K-Means Silhouette Skoru : {results['K-Means Silhouette']:.4f}")
print(f"Hiyerarşik Silhouette Skoru : {results['Hierarchical Silhouette']:.4f}")
print("-" * 40)

Modeller PCA verisi üzerinde başarıyla eğitildi.

----------------------------------------
K-Means Silhouette Skoru (Sadek): 0.2327
Hiyerarşik Silhouette Skoru (Sadek): 0.2037
----------------------------------------


In [5]:
# --- Modellerin Diske Kaydedilmesi  ---

if not os.path.exists('saved_models'):
    os.makedirs('saved_models')

print("Dosyalar kaydediliyor...")

# 1. Modelleri Kaydet
joblib.dump(kmeans_model, 'saved_models/kmeans_model_sadek.pkl')
joblib.dump(agg_model, 'saved_models/agg_model_sadek.pkl')

# 2. Scaler'ı Kaydet (Veriyi aynı standarda getirmek için şart)
joblib.dump(scaler, 'saved_models/scaler_sadek.pkl')

# 3. PCA Nesnesini Kaydet (ÖNEMLİ)
# Yeni gelen veriyi aynı matematiksel düzleme indirmek için PCA modeline ihtiyacımız var.
joblib.dump(pca, 'saved_models/pca_object_sadek.pkl')

print("BAŞARILI: Tüm dosyalar 'saved_models' klasörüne kaydedildi.")
print("Kaydedilen dosyalar:")
print("- kmeans_model_sadek.pkl")
print("- agg_model_sadek.pkl")
print("- scaler_sadek.pkl")
print("- pca_object_sadek.pkl (Boyut indirgeme aracı)")

Dosyalar kaydediliyor...
BAŞARILI: Tüm dosyalar 'saved_models' klasörüne kaydedildi.
Kaydedilen dosyalar:
- kmeans_model_sadek.pkl
- agg_model_sadek.pkl
- scaler_sadek.pkl
- pca_object_sadek.pkl (Boyut indirgeme aracı)
